In [21]:
import cv2
import numpy as np
from PIL import Image, ImageDraw, ImageColor
import fitz  # PyMuPDF

def hex_to_hsv(hex_color):
    rgb = ImageColor.getcolor(hex_color, "RGB")
    r, g, b = [x / 255.0 for x in rgb]
    hsv = cv2.cvtColor(np.uint8([[[b * 255, g * 255, r * 255]]]), cv2.COLOR_BGR2HSV)[0][0]
    return hsv

def enhance_table_lines_from_pdf_hq(pdf_path, output_path, page_number=0, dpi=300):
    """
    Enhances vertical column separators, preserving image quality.

    Args:
        pdf_path: Path to the PDF.
        output_path: Path to save the image (use .png).
        page_number: Page to process (0-indexed).
        dpi: Resolution for rendering the PDF page.
    """
    doc = fitz.open(pdf_path)
    page = doc[page_number]

    # Get Pixmap at specified DPI (this is KEY for quality)
    pix = page.get_pixmap(dpi=dpi)  # High resolution!
    img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
    img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)  # Convert PIL Image to OpenCV format
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # --- 1. Green Row Detection ---
    target_hsv = hex_to_hsv("#D8EDCF")
    tolerance = 10
    lower_green = np.array([max(0, target_hsv[0] - tolerance), 50, 50])
    upper_green = np.array([min(179, target_hsv[0] + tolerance), 255, 255])
    green_mask = cv2.inRange(hsv, lower_green, upper_green)
    h_proj_green = np.sum(green_mask, axis=1)
    green_row_indices = np.where(h_proj_green > 0)[0]

    if len(green_row_indices) > 0:
        top_boundary = green_row_indices[0]
        bottom_boundary = green_row_indices[-1]
    else:
        print("No green rows detected.")
        return
      

    # --- 2. Header Row Detection ---
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert once, reuse
    header_region = gray[0:top_boundary, :]
    #_, binary_header = cv2.threshold(header_region, 180, 255, cv2.THRESH_BINARY_INV)
    _, binary_header = cv2.threshold(
    header_region, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
    h_proj_header = np.sum(binary_header, axis=1)
    header_bottom = 0
    for i in range(len(h_proj_header) - 1, 0, -1):
        if h_proj_header[i] > 40:
            header_bottom = i
            break

    # --- 3. Adaptive Thresholding (within table, on GRAYSCALE) ---
    table_region = gray[top_boundary:bottom_boundary, :]
    thresh_table = cv2.adaptiveThreshold(table_region, 255, cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY_INV, 11, 3)

    # --- 4. Hough Lines (within table) ---
    lines = cv2.HoughLinesP(thresh_table, 1, np.pi / 180, threshold=775,
                            minLineLength=10, maxLineGap=8)

    # --- 5. Filter Vertical Lines ---
    vertical_lines = []
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            if abs(x2 - x1) < 5:
                # Adjust coordinates *back* to the full image
                vertical_lines.append((x1, y1 + top_boundary, x2, y2 + top_boundary))

    # --- 6. Draw Lines (Directly on the OpenCV Image) ---
    # Draw lines directly on the BGR image (no PIL conversion yet)
    #print(vertical_lines)
    for x1, y1, x2, y2 in vertical_lines:
        #cv2.line(img, (x1, header_bottom + 200), (x2, bottom_boundary - 175), 
        cv2.line(img, (x1, header_bottom), (x2, bottom_boundary), (100, 100, 100), 1)

    # --- 7. Convert to PIL and Save (ONE conversion) ---
    img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))  # Convert back to RGB
    img_pil.save(output_path)




In [22]:
for week in [1,2,3,4,12,22,26,32,42,52]:
    enhance_table_lines_from_pdf_hq(f"PDFs/W{week}.pdf", f"Lines_W{week}.png", page_number=3, dpi=300)


# Main script

In [45]:
import cv2
import numpy as np
from PIL import Image, ImageColor
import fitz  # PyMuPDF

def hex_to_hsv(hex_color):
    rgb = ImageColor.getcolor(hex_color, "RGB")
    r, g, b = [x / 255.0 for x in rgb]
    hsv = cv2.cvtColor(np.uint8([[[b * 255, g * 255, r * 255]]]), cv2.COLOR_BGR2HSV)[0][0]
    return hsv

def enhance_table_lines_from_pdf_hq(pdf_path, output_path, page_number=0, dpi=300):
    """
    Enhances vertical column separators and draws horizontal lines at
    top boundary, bottom boundary, and header bottom.

    Args:
        pdf_path (str): Path to the PDF.
        output_path (str): Path to save the image (use .png).
        page_number (int): Page to process (0-indexed).
        dpi (int): Resolution for rendering the PDF page.
    """
    doc = fitz.open(pdf_path)
    page = doc[page_number]

    # 1. Render the PDF page at high DPI
    pix = page.get_pixmap(dpi=dpi) 
    img_pil = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
    # Convert PIL Image to OpenCV BGR
    img = cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)

    # 2. Convert to HSV to detect green rows
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    target_hsv = hex_to_hsv("#D8EDCF")
    tolerance = 5
    lower_green = np.array([max(0, target_hsv[0] - tolerance), 50, 50])
    upper_green = np.array([min(179, target_hsv[0] + tolerance), 255, 255])
    green_mask = cv2.inRange(hsv, lower_green, upper_green)
    h_proj_green = np.sum(green_mask, axis=1)
    green_row_indices = np.where(h_proj_green > 0)[0]

    if len(green_row_indices) == 0:
        print("No green rows detected.")
        return

    top_boundary = green_row_indices[0]
    bottom_boundary = green_row_indices[-1]

    # 3. Header Row Detection (just above top_boundary)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    header_region = gray[:top_boundary, :]

    # Use Otsu’s threshold instead of fixed 180
    _, binary_header = cv2.threshold(header_region, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
    h_proj_header = np.sum(binary_header, axis=1)

    header_bottom = 0
    for i in range(len(h_proj_header) - 1, 0, -1):
        if h_proj_header[i] > 40:
            header_bottom = i
            break

    # 4. Adaptive Thresholding in the table region
    table_region = gray[top_boundary:bottom_boundary, :]
    thresh_table = cv2.adaptiveThreshold(
        table_region, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 11, 3
    )

    # 5. Hough Lines to find vertical lines
    # Increase or decrease threshold=775 if you get too few/many lines
    lines = cv2.HoughLinesP(thresh_table, 1, np.pi / 180, threshold=600,
                            minLineLength=10, maxLineGap=8)
    vertical_lines = []
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            # Check near-vertical
            if abs(x2 - x1) < 5:
                # Adjust back to full image coordinates
                vertical_lines.append((x1, y1 + top_boundary, x2, y2 + top_boundary))

    # 6. Draw lines on the OpenCV image
    # A) Draw the vertical lines from header_bottom to bottom_boundary
    for x1, y1, x2, y2 in vertical_lines:
        cv2.line(img, (x1, 350), (x2, bottom_boundary-185), (100, 100, 100), 1)
   
    print(header_bottom)
    # B) Draw horizontal lines at:
    #    - top_boundary
    #    - bottom_boundary
    #    - header_bottom
    # We'll draw them across the full width of the image
    height, width = img.shape[:2]
    # color is (100,100,100); thickness=1 or 2 as you prefer
    # Top boundary (green)
    cv2.line(img, (0, top_boundary), (width, top_boundary), (0, 255, 0), 2)

    # Bottom boundary (green)
    cv2.line(img, (0, bottom_boundary), (width, bottom_boundary), (0, 255, 0), 2)

    # Header bottom (red)
    cv2.line(img, (0, header_bottom), (width, header_bottom), (0, 0, 255), 2)

    # 7. Convert back to PIL and save
    output_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    output_pil.save(output_path)

    print(f"Saved enhanced table to: {output_path}")


In [46]:
for week in [1,2,3,4,12,22,26,32,42,52]:
    enhance_table_lines_from_pdf_hq(f"PDFs/W{week}.pdf", f"Lines_W{week}_boundaries.png", page_number=3, dpi=300)

515
Saved enhanced table to: Lines_W1_boundaries.png
405
Saved enhanced table to: Lines_W2_boundaries.png
407
Saved enhanced table to: Lines_W3_boundaries.png
410
Saved enhanced table to: Lines_W4_boundaries.png
402
Saved enhanced table to: Lines_W12_boundaries.png
407
Saved enhanced table to: Lines_W22_boundaries.png
408
Saved enhanced table to: Lines_W26_boundaries.png
427
Saved enhanced table to: Lines_W32_boundaries.png
430
Saved enhanced table to: Lines_W42_boundaries.png
399
Saved enhanced table to: Lines_W52_boundaries.png


# 15% left

In [19]:
import cv2
import numpy as np
from PIL import Image, ImageColor
import fitz  # PyMuPDF

def hex_to_hsv(hex_color):
    rgb = ImageColor.getcolor(hex_color, "RGB")
    r, g, b = [x / 255.0 for x in rgb]
    hsv = cv2.cvtColor(
        np.uint8([[[b * 255, g * 255, r * 255]]]),
        cv2.COLOR_BGR2HSV
    )[0][0]
    return hsv

def enhance_table_lines_from_pdf_hq(pdf_path, output_path, page_number=0, dpi=300):
    """
    Enhances vertical column separators and draws horizontal lines at
    top boundary, bottom boundary, and header bottom.

    Args:
        pdf_path (str): Path to the PDF.
        output_path (str): Path to save the image (use .png).
        page_number (int): Page to process (0-indexed).
        dpi (int): Resolution for rendering the PDF page.
    """
    doc = fitz.open(pdf_path)
    page = doc[page_number]

    # 1. Render the PDF page at high DPI
    pix = page.get_pixmap(dpi=dpi) 
    img_pil = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
    # Convert PIL Image to OpenCV BGR
    img = cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)

    # 2. Convert to HSV & detect green rows
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    height, width = img.shape[:2]
    
    # We'll only check the left 15% of the page's width
    left_15_width = int(width * 0.15)

    target_hsv = hex_to_hsv("#D8EDCF")
    tolerance = 15
    lower_green = np.array([max(0, target_hsv[0] - tolerance), 50, 50])
    upper_green = np.array([min(179, target_hsv[0] + tolerance), 255, 255])
    green_mask = cv2.inRange(hsv, lower_green, upper_green)

    # Sum only the left 15% columns
    green_mask_left = green_mask[:, :left_15_width]
    h_proj_green = np.sum(green_mask_left, axis=1)
    green_row_indices = np.where(h_proj_green > 0)[0]

    if len(green_row_indices) == 0:
        print("No green rows detected.")
        return

    top_boundary = green_row_indices[0]
    bottom_boundary = green_row_indices[-1]

    # 3. Header Row Detection (just above top_boundary)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    header_region = gray[:top_boundary, :]

    # Use Otsu’s threshold for the header
    _, binary_header = cv2.threshold(
        header_region, 0, 255,
        cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU
    )
    h_proj_header = np.sum(binary_header, axis=1)

    header_bottom = 0
    for i in range(len(h_proj_header) - 1, 0, -1):
        if h_proj_header[i] > 40:
            header_bottom = i
            break

    # 4. Adaptive Thresholding in the table region
    table_region = gray[top_boundary:bottom_boundary, :]
    thresh_table = cv2.adaptiveThreshold(
        table_region,
        255,
        cv2.ADAPTIVE_THRESH_MEAN_C,
        cv2.THRESH_BINARY_INV,
        11,
        3
    )

    # 5. Hough Lines to find vertical lines
    lines = cv2.HoughLinesP(thresh_table, 1, np.pi / 180,
                            threshold=775, minLineLength=10, maxLineGap=8)
    vertical_lines = []
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            if abs(x2 - x1) < 5:  # near-vertical
                # Adjust back to full image coordinates
                vertical_lines.append((x1, y1 + top_boundary, x2, y2 + top_boundary))

    # 6. Draw lines on the OpenCV image
    # A) Draw the vertical lines from header_bottom to bottom_boundary
    for x1, y1, x2, y2 in vertical_lines:
        cv2.line(img, (x1, header_bottom), (x2, bottom_boundary), (100, 100, 100), 1)

    # B) Draw horizontal lines at top, bottom, and header_bottom
    cv2.line(img, (0, top_boundary), (width, top_boundary), (0, 255, 0), 2)     # green
    cv2.line(img, (0, bottom_boundary), (width, bottom_boundary), (0, 255, 0), 2) # green
    cv2.line(img, (0, header_bottom), (width, header_bottom), (0, 0, 255), 2)  # red

    # 7. Convert back to PIL and save
    output_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    output_pil.save(output_path)

    print(f"Saved enhanced table to: {output_path}")


In [20]:
for week in [1,2,3,4,12,22,26,32,42,52]:
    enhance_table_lines_from_pdf_hq(f"PDFs/W{week}.pdf", f"Lines_W{week}_boundaries.png", page_number=3, dpi=300)

Saved enhanced table to: Lines_W1_boundaries.png
Saved enhanced table to: Lines_W2_boundaries.png
Saved enhanced table to: Lines_W3_boundaries.png
Saved enhanced table to: Lines_W4_boundaries.png
Saved enhanced table to: Lines_W12_boundaries.png
Saved enhanced table to: Lines_W22_boundaries.png
Saved enhanced table to: Lines_W26_boundaries.png
Saved enhanced table to: Lines_W32_boundaries.png
Saved enhanced table to: Lines_W42_boundaries.png
Saved enhanced table to: Lines_W52_boundaries.png


# PDF Sourcing

In [47]:
!pip install requests beautifulsoup4

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [48]:
import os
import requests
from bs4 import BeautifulSoup

def download_lassa_pdfs():
    # The base URL that hosts the PDFs
    base_url = "https://ncdc.gov.ng"
    # The specific page that lists all the Lassa fever situation reports
    list_page_url = (
        "https://ncdc.gov.ng/diseases/sitreps/?cat=5&name=An%20update%20of%20Lassa%20fever%20outbreak%20in%20Nigeria"
    )
    
    # Create a local folder to store the PDFs
    os.makedirs("pdfs", exist_ok=True)
    
    # 1. Fetch the HTML
    print(f"Fetching list page: {list_page_url}")
    response = requests.get(list_page_url)
    response.raise_for_status()  # raise an error if the HTTP request failed
    
    # 2. Parse the HTML
    soup = BeautifulSoup(response.text, "html.parser")
    
    # The table is inside <tbody>. Each row has multiple <td>, 
    # and the third <td> has the <a> with the PDF link
    table_body = soup.find("tbody")
    if not table_body:
        print("Could not find <tbody> on the page.")
        return
    
    rows = table_body.find_all("tr")
    if not rows:
        print("No <tr> found inside <tbody>.")
        return

    # For stats
    total_found = 0
    total_downloaded = 0
    
    for row in rows:
        cells = row.find_all("td")
        if len(cells) < 3:
            # We expect 3 <td> in each row: (1) index, (2) description, (3) the PDF link
            continue
        
        # The PDF link is in the third cell; let's get the <a>:
        link_tag = cells[2].find("a", href=True)
        if not link_tag:
            continue
        
        # The PDF URL is relative, e.g. "/themes/common/files/sitreps/..."
        # We need to prepend https://ncdc.gov.ng
        pdf_url = link_tag["href"]
        if pdf_url.startswith("/"):
            pdf_url = base_url + pdf_url
        
        # The "download" attribute often has the suggested filename
        # or we can parse from the final part of the URL
        download_name = link_tag.get("download")  # e.g. "An update of Lassa fever ... .pdf"
        
        if not download_name:
            # Fallback: parse the filename from the URL
            download_name = pdf_url.split("/")[-1]

        # Clean up the download name if needed
        download_name = download_name.replace(" ", "_")

        total_found += 1
        # 3. Download the PDF
        # We'll skip if it already exists. Or you can overwrite by removing the check.
        local_path = os.path.join("pdfs", download_name)
        if os.path.exists(local_path):
            print(f"Already downloaded: {download_name}")
            continue
        
        print(f"Downloading {pdf_url} -> {local_path}")
        try:
            pdf_response = requests.get(pdf_url)
            pdf_response.raise_for_status()
            with open(local_path, "wb") as f:
                f.write(pdf_response.content)
            total_downloaded += 1
        except Exception as e:
            print(f"Failed to download {pdf_url}: {e}")

    print(f"Found {total_found} PDF links total. Downloaded {total_downloaded} new PDFs.")

if __name__ == "__main__":
    download_lassa_pdfs()


/Users/arturtrebski/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Fetching list page: https://ncdc.gov.ng/diseases/sitreps/?cat=5&name=An%20update%20of%20Lassa%20fever%20outbreak%20in%20Nigeria
Failed to download https://ncdc.gov.ng/themes/common/files/sitreps/7bff918b81a8e610200b879d929995da.pdf: 404 Client Error: Not Found for url: https://ncdc.gov.ng/themes/common/files/sitreps/7bff918b81a8e610200b879d929995da.pdf
Failed to download https://ncdc.gov.ng/themes/common/files/sitreps/6ce166b15bffc6986d356448d48d6a34.pdf: 404 Client Error: Not Found for url: https://ncdc.gov.ng/themes/common/files/sitreps/6ce166b15bffc6986d356448d48d6a34.pdf
Failed to download https://ncdc.gov.ng/themes/common/files/sitreps/c961c37b5cf7d7f010f0b991722f89ba.pdf: 404 Client Error: Not Found for url: https://ncdc.gov.ng/themes/common/files/sitreps/c961c37b5cf7d7f010f0b991722f89ba.pdf
Found 400 PDF links total. Downloaded 397 new PDFs.


In [49]:
import os

pdf_files = os.listdir('PDFs')
print(pdf_files)

['An_update_of_Lassa_fever_outbreak_in_Nigeria_230323_13.pdf', 'An_update_of_Lassa_fever_outbreak_in_Nigeria_080122_2.pdf', 'An_update_of_Lassa_fever_outbreak_in_Nigeria_260321_13.pdf', 'An_update_of_Lassa_fever_outbreak_in_Nigeria_010718_27.pdf', 'An_update_of_Lassa_fever_outbreak_in_Nigeria_080623_24.pdf', 'An_update_of_Lassa_fever_outbreak_in_Nigeria_230319_12.pdf', 'An_update_of_Lassa_fever_outbreak_in_Nigeria_110218_7.pdf', 'An_update_of_Lassa_fever_outbreak_in_Nigeria_140619_24.pdf', 'An_update_of_Lassa_fever_outbreak_in_Nigeria_040523_19.pdf', 'An_update_of_Lassa_fever_outbreak_in_Nigeria_050719_27.pdf', 'An_update_of_Lassa_fever_outbreak_in_Nigeria_110724_28.pdf', 'An_update_of_Lassa_fever_outbreak_in_Nigeria_110120_2.pdf', 'An_update_of_Lassa_fever_outbreak_in_Nigeria_250125_4.pdf', 'An_update_of_Lassa_fever_outbreak_in_Nigeria_300524_22.pdf', 'An_update_of_Lassa_fever_outbreak_in_Nigeria_040720_27.pdf', 'An_update_of_Lassa_fever_outbreak_in_Nigeria_160524_20.pdf', 'An_update_

In [ ]:
import os
import re
from pathlib import Path

def rename_lassa_files(folder_path):
    """
    Renames 'An_update_of_Lassa_fever_outbreak_in_Nigeria_041124_45.pdf'
    to 'Nigeria_04_Nov_24_W45.pdf', extracting day=04, month=11 => 'Nov', year=24,
    and the week number 45.
    
    Args:
        folder_path (str): Path to the folder that contains the PDF files.
    """
    # For mapping month number to short name
    month_map = {
        "01": "Jan", "02": "Feb", "03": "Mar", "04": "Apr",
        "05": "May", "06": "Jun", "07": "Jul", "08": "Aug",
        "09": "Sep", "10": "Oct", "11": "Nov", "12": "Dec",
    }

    folder = Path(folder_path)
    for file_path in folder.iterdir():
        if not file_path.is_file():
            continue
        if not file_path.suffix.lower() == ".pdf":
            continue
        
        old_name = file_path.name
        # Example old_name: "An_update_of_Lassa_fever_outbreak_in_Nigeria_041124_45.pdf"
        
        # 1) Split on underscores
        parts = old_name.split("_")
        # e.g. ["An","update","of","Lassa","fever","outbreak","in","Nigeria","041124","45.pdf"]
        
        if len(parts) < 9:
            # If the file name doesn't match the expected pattern, skip it
            print(f"Skipping file (unrecognized pattern): {old_name}")
            continue
        
        # 2) The date chunk is parts[8] like "041124"
        date_str = parts[8]  # "041124"
        
        # 3) The week chunk is in parts[9], but includes ".pdf" at the end, e.g. "45.pdf"
        week_str_pdf = parts[9]  # "45.pdf"
        # Remove ".pdf" from the end
        if week_str_pdf.endswith(".pdf"):
            week_str = week_str_pdf.replace(".pdf", "")
        else:
            print(f"Skipping file (no .pdf in last part): {old_name}")
            continue
        
        # 4) date_str should be 6 characters: DDMMYY
        if len(date_str) != 6:
            print(f"Skipping file (date string not 6 chars): {old_name}")
            continue
        dd = date_str[0:2]   # "04"
        mm = date_str[2:4]   # "11"
        yy = date_str[4:6]   # "24"
        
        # 5) Convert mm => month name
        month_name = month_map.get(mm, "???" )  # fallback "???"
        
        # 6) Build new name
        # e.g. "Nigeria_04_Nov_24_W45.pdf"
        new_name = f"Nigeria_{dd}_{month_name}_{yy}_W{week_str}.pdf"
        
        new_path = folder / new_name
        # 7) Rename the file
        print(f"Renaming:\n  {old_name}\n-> {new_name}\n")
        file_path.rename(new_path)

# Example usage:
if __name__ == "__main__":
    rename_lassa_files("PDFs")
